### Dynamics
```
Dynamics defines a mapping between force and acceleration aka Newton's second law of motion (i.e., equation of motion) 
```
- Forward kinematics -> Contact solving -> Compute acceleration -> Inetgration

In our implementation, dynamic simulation is done by `env.step` and internally `mujoco.mj_step` is called

In [1]:
import sys,mujoco
sys.path.append('../package/helper/')
sys.path.append('../package/mujoco_usage/')
from mujoco_parser import *
from slider import * # <= slider control
from utility import *
print ("MuJoCo:[%s]"%(mujoco.__version__))

MuJoCo:[3.1.1]


#### Slider control of `UR5e` using dynamic simulation

In [2]:
xml_path = '../asset/tabletop_manipulation/scene_ur5e_objects.xml'
env = MuJoCoParserClass(name='UR5e',rel_xml_path=xml_path,verbose=True)

init_ctrl = env.get_ctrl(ctrl_names=env.ctrl_names)
sliders = MultiSliderClass(
    n_slider      = env.n_ctrl,
    title         = 'Sliders for [%s] Control'%(env.name),
    window_width  = 600,
    window_height = 350,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 300,
    label_texts   = env.ctrl_names,
    slider_mins   = env.ctrl_mins,
    slider_maxs   = env.ctrl_maxs,
    slider_vals   = init_ctrl,
    verbose       = False,
)
# Arrange objects
env.reset(step=True)
obj_names = env.get_body_names(prefix='obj_')
n_obj = len(obj_names)
obj_xyzs = sample_xyzs(
    n_sample  = n_obj,
    x_range   = [+0.6,+1.0],
    y_range   = [-0.45,+0.45],
    z_range   = [0.8,0.81],
    min_dist  = 0.2,
    xy_margin = 0.0
)
for obj_idx in range(n_obj):
    env.set_p_base_body(body_name=obj_names[obj_idx],p=obj_xyzs[obj_idx,:])
    env.set_R_base_body(body_name=obj_names[obj_idx],R=np.eye(3,3))
env.set_geom_color(body_names_to_color=obj_names,rgba_list=get_colors(n_obj))
# Loop
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    # Update
    sliders.update()
    env.step(ctrl=sliders.get_slider_values())
    # Render
    if env.loop_every(tick_every=50):
        env.plot_time()
        env.plot_joint_axis(axis_len=0.025,axis_r=0.005) # revolute joints
        env.plot_links_between_bodies(rgba=(0,0,0,1),r=0.001) # link information
        env.plot_contact_info()
        env.render()
# Close slider
sliders.close()
print ("Done.")

name:[UR5e] dt:[0.002] HZ:[500]
n_qpos:[33] n_qvel:[30] n_qacc:[30] n_ctrl:[7]

n_body:[27]
 [0/27] [world] mass:[0.00]kg
 [1/27] [front_object_table] mass:[1.00]kg
 [2/27] [side_object_table] mass:[1.00]kg
 [3/27] [ur_base] mass:[4.00]kg
 [4/27] [ur_shoulder_link] mass:[3.70]kg
 [5/27] [ur_upper_arm_link] mass:[8.39]kg
 [6/27] [ur_forearm_link] mass:[2.27]kg
 [7/27] [ur_wrist_1_link] mass:[1.22]kg
 [8/27] [ur_wrist_2_link] mass:[1.22]kg
 [9/27] [ur_wrist_3_link] mass:[0.19]kg
 [10/27] [ur_tcp_link] mass:[0.00]kg
 [11/27] [ur_camera_mount] mass:[0.09]kg
 [12/27] [ur_d435i] mass:[0.07]kg
 [13/27] [ur_rg2_gripper_base_link] mass:[0.20]kg
 [14/27] [ur_camera_center] mass:[0.00]kg
 [15/27] [ur_rg2_gripper_finger1_finger_link] mass:[0.01]kg
 [16/27] [ur_rg2_gripper_finger1_inner_knuckle_link] mass:[0.01]kg
 [17/27] [ur_rg2_gripper_finger1_finger_tip_link] mass:[0.01]kg
 [18/27] [ur_rg2_gripper_finger2_finger_link] mass:[0.01]kg
 [19/27] [ur_rg2_gripper_finger2_inner_knuckle_link] mass:[0.01

#### Parse `Unitree G1`

In [3]:
xml_path = '../asset/unitree_g1/scene_g1.xml'
env = MuJoCoParserClass(name='Unitree G1',rel_xml_path=xml_path,verbose=True)

name:[Unitree G1] dt:[0.002] HZ:[500]
n_qpos:[44] n_qvel:[43] n_qacc:[43] n_ctrl:[37]

n_body:[39]
 [0/39] [world] mass:[0.00]kg
 [1/39] [pelvis] mass:[2.86]kg
 [2/39] [left_hip_pitch_link] mass:[1.30]kg
 [3/39] [left_hip_roll_link] mass:[1.45]kg
 [4/39] [left_hip_yaw_link] mass:[2.05]kg
 [5/39] [left_knee_link] mass:[2.25]kg
 [6/39] [left_ankle_pitch_link] mass:[0.07]kg
 [7/39] [left_ankle_roll_link] mass:[0.39]kg
 [8/39] [right_hip_pitch_link] mass:[1.30]kg
 [9/39] [right_hip_roll_link] mass:[1.45]kg
 [10/39] [right_hip_yaw_link] mass:[2.05]kg
 [11/39] [right_knee_link] mass:[2.25]kg
 [12/39] [right_ankle_pitch_link] mass:[0.07]kg
 [13/39] [right_ankle_roll_link] mass:[0.39]kg
 [14/39] [torso_link] mass:[7.52]kg
 [15/39] [left_shoulder_pitch_link] mass:[0.71]kg
 [16/39] [left_shoulder_roll_link] mass:[0.64]kg
 [17/39] [left_shoulder_yaw_link] mass:[0.71]kg
 [18/39] [left_elbow_pitch_link] mass:[0.60]kg
 [19/39] [left_elbow_roll_link] mass:[0.51]kg
 [20/39] [left_zero_link] mass:[0.05

#### Animate free-fall motion

In [4]:
env.reset(step=True)
env.init_viewer()
while env.is_viewer_alive():
    env.step()
    env.render()
env.close_viewer()
print ("Done.")

Done.


#### Animate free-fall motion with other information

In [5]:
env.reset(step=True)
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    env.step()
    if env.loop_every(tick_every=10):
        env.plot_T()
        env.plot_time() # time
        env.plot_joint_axis(axis_len=0.025,axis_r=0.005) # revolute joints
        env.plot_contact_info() # contact information
        env.render()
env.close_viewer()
print ("Done.")

Done.
